Abstract and explination here

Create list of sections

1. DATA CLEANING
2. CHOOSING THE STOCKS

define process overview, strategies, goals

In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import threading
import time
from queue import Queue

## Group Assignment
### Team Number: 05
### Team Member Names: Piero Camposeo, Sathun Suthakaran, Ishaan Bansal
### Team Strategy Chosen: SAFE

In [2]:
#Read in csv file of tickers
csv_file = "Tickers.csv"

# Make values into a list
tickers = pd.read_csv(csv_file)
ticker_list = tickers.iloc[:, 0].values.tolist()
ticker_list.insert(0, tickers.columns[0])

In [3]:
#Consumes desired amount of trading days within a month, a dataframe of days, a dstaframe of months, and a list of months that cooresponds to the months
#    on the dataframe, outputs the monthly dataframe with only months that have 'daycount' amount of trading days for that stock
#Note: Function will cover up to the last date. For example, the function will not produce October for a dataframe which ends on October 1st.
#   In our code, we simply passed a days dataframe that ends one month after the months dataframe does (*)
def df_with_valid_months(daycount, df_days, df_months, months):
    for month in range (len(months)):
        if len(df_days.filter(like = months[month], axis = 0)) < daycount:
            df_months.drop(df_months.index[month], axis = 0, inplace = True) #Drop any months with not enough trading days
            
    return(df_months)

#Consumes historical data for a stock, outputs average volume
def average_volume(df_hist):
    volume = df_hist['Volume'].dropna().values.tolist()
    vol_av = np.mean(volume)

    return(vol_av)


## 1. DATA CLEANING: 

This cell is where the data cleaning happens.

THE GOAL:

•	Ignore any tickers that do not reference a valid stock denominated in USD, i.e., only use US listed stocks.

•	Use only tickers that reference stocks which have an average monthly volume of at least 200,000 shares, as calculated based on the time interval of January 01, 2022 to October 31, 2022. A month is defined as a calendar month. We will drop any month that does not have at least 20 trading days.

THE PROCESS:

Firstly, we have chosen to implement multithreading, in order to speed up the process. The data cleaning is the most time-consuming code in the program to run. Due to this, we have two main functions (currency_check and volume_check), running on two seperate threads, c and v. Rigorous testing done by our group has shown that the currency check is the most time-consuming function, taking around 4-12 minutes (with a sample size of 62 tickers) (as opposed to volume_check, which takes 1-2 minutes) depending on the system on which the code is being run, as well as other (seemingly random) factors. Most of that time is spent retrieving .info information from each respective ticker. As there is no other (trusted) way to do this, we hypothesize that this implementation is about as fast as possible

We use Queues to store appropriate variables, so that they can be retrieved later by the function.

Our code checks each of the two criterion mentioned (one for each function), and creates two lists. Each list contains tickers which reference stocks that have satisfied the criteria of the functions. As we want stocks that fit both criterion, our final list of (valid) tickers is the set intersection of the two lists which each function produces.


In [5]:
#dates for volume calculation, end date for valid months function (see (*))
vol_date_s = '2022-01-01'
vol_date_e = '2022-10-02'
endDForValidMonthsFunct = str((pd.to_datetime(vol_date_e) + relativedelta(months = 1)).strftime("%Y-%m-%d"))

#for df_with_valid_months function, creates a series of months from vol_date_s to vol_date_e
months = pd.date_range(vol_date_s, vol_date_e, freq='MS').strftime("%Y-%m").tolist()

#As outlined in assignment information
minimumMonthlyTradingDays = 20
minimum_volume = 200000
required_denomination = 'USD'

#Here we will put valid stocks that fit our criteria
tickers = []

currency_valid = []
volume_valid = []

#Calling on Queue class
queueC = Queue()
queueV = Queue()

#Consumes a list of tickers, checks if they are denominated in 'required_denomination'.
def currency_check(ticker_list):
    valid = []

    if len(ticker_list) <= 10:

        #Retrieve financial currency from each ticker, if it is 'required_denomination', it is a valid ticker. Else, we don't care so we set it as nan
        for ticker in ticker_list:
            tick = yf.Ticker(ticker)
            try:
                currency = tick.info['financialCurrency']
            except:
                currency = 'nan'
        
            if currency == required_denomination:
                valid.append(ticker)

        #Pass our valid tickers to the QueueC (Queue Currency), so that they may be called on later
        queueC.put(valid)
    else:
        ticker_split = np.array_split(ticker_list, 5)

        #queueCSub = Queue()

        cSub1 = threading.Thread(target = currency_check, args = (ticker_split[0],))
        cSub2 = threading.Thread(target = currency_check, args = (ticker_split[1],))
        cSub3 = threading.Thread(target = currency_check, args = (ticker_split[2],))
        cSub4 = threading.Thread(target = currency_check, args = (ticker_split[3],))
        cSub5 = threading.Thread(target = currency_check, args = (ticker_split[4],))

        cSub1.start()
        cSub2.start()
        cSub3.start()
        cSub4.start()
        cSub5.start()

        cSub1.join()
        cSub2.join()
        cSub3.join()
        cSub4.join()
        cSub5.join()
 
#Consumes a list of tickers, checks if they have an average monthly volume of 'minimum_volume' or more over each month with 
# 'minimumMonthlyTradingDays' or more trading days
def volume_check(ticker_list):
    valid = []
    
    for ticker in ticker_list:
        tick = yf.Ticker(ticker)
        
        #Check for Delisted ticker + Creating histories. If 'check' returns an error, the code continues. Without this try/except,
        #Error code for delisted stock appears twice
        try:
            tick_hist = tick.history(start = vol_date_s, end  = vol_date_e, interval = '1mo').dropna()
            check = tick_hist.iloc[0,0]
            #Custom dataframe for valid months check (daily, ending one month after monthly dataframe)
            tickHistForValidMonthsCheck = tick.history(start = vol_date_s, end  = endDForValidMonthsFunct, interval = '1d').dropna()
 
            #Dataframe with valid months only
            tickHistWithValidMonths = df_with_valid_months(minimumMonthlyTradingDays, tickHistForValidMonthsCheck, tick_hist, months)
       
        except:
            pass
            
        #Call on average_volume function to compute and average volume
        monthly_average_volume = 0 #Initialize this to 0 to prevent errors
        monthly_average_volume = average_volume(tickHistWithValidMonths)
        
        #Check, append to valid if criteria is fit
        if monthly_average_volume >= minimum_volume:
            valid.append(ticker)

    #Pass our valid tickers to the QueueV (Queue Volume), so that they may be called on later
    queueV.put(valid)
        
#Creating threads for currency_check and volume_check function
c = threading.Thread(target = currency_check, args = (ticker_list,))
v = threading.Thread(target = volume_check, args = (ticker_list,))

#Start thread executions
c.start()
v.start()

v.join()
volume_valid = queueV.get()

#Wait for threads to finish and retrieve valid tickers from respective queues
c.join()
while not queueC.empty():
    currency_valid.append(queueC.get())

currency_valid = [item for sublist in currency_valid for item in sublist]

#Valid tickers are the intersection of the two lists (Must have particular denomination and average monthly volume as outlined in assignment)
tickers = sorted([value for value in currency_valid if value in volume_valid])

print(f"\nValid Tickers: {tickers}")

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted

Valid Tickers: ['AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'GM', 'GOOG', 'JPM', 'KMI', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'MS', 'MSFT', 'NEE', 'NKE', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


## 2. CHOOSING THE STOCKS:

The following cells are where we choose the stocks to be used for the final portfolio.

THE GOAL:
Since our overall goal is to have a portfolio which finishes as close to its starting value as possible, we naturally want a diversified and stable selection of stocks. With this in mind, we decided to choose as many stocks as allowed (so as to avoid unsystematic risk), with stocks chosen being the least volatile out of the available selection. In summary, we want:
•	(if there are less than 25 tickers to choose from) every (valid) ticker available <b>OR</b> 25 tickers (maximum allowed as outlined in assignment information) comprised of:
•	The least volatile stocks available
    •   Note: Volatility, as we define it here, is represented by the standard deviation of daily returns (over a 5 month period, specifically)

THE PROCESS:

.......

In [7]:
#Sathun

In [8]:
#####EXAMPLE##########

#Example pr_df, pretend numbers are percent changes (They'll be smaller and there will be many more of them in the actual thing)

dict = {'APPL': [.3,1,2,3,1,2,3,4,5,6,4,3,2,3,3,4,5,3,2,4,4],
        'PG': [.3,3,2,3,1,2,3,4,2,6,4,3,0.1,3,3,4,9,3,2,4,4],
        'T': [.3,1,2,3,1,2,3,4,2,6,4,3,0.3,3,3,4,5,3,1,4,4],
        'F': [.3,1,2,3,3,2,3,2,2,6,4,3,2,3,3,1,5,3,3,4,4],
        'JNJ': [1,1,0.6,3,1,2,3,4,5,6,4,3,2,3,3,4,5,3,2,4,4],
        'BAC': [.3,0.8,2,3,1,2,3,4,5,6,4,3,2,3,3,4,5,3,2,4,4]}

pr_df = pd.DataFrame(dict)

In [9]:
max_portfolios = 25

#Ishaan FOR LATER: WRITE WHY VOLATILITY IS WITHOUT TIME PERIOD IN EXPLANATION SOMEWHERE (LOOK AT VOLATILITY FORMULA)
#     Piero's note: Just say "volatility, represented by the average standard deviation of daily returns over a 5 month period"

# Consumes a Dataframe of stocks and their respective daily % returns and produces a list of the 'max_portfolios' least volatile stocks, or if there's
# Less than 'max_portfolios' stocks to choose from, produces a list of all of the stocks. Also prints dataframe of final stocks and their respected
# volatility, as well as the number of stocks in the final portfolio
def lowest_volatile_stocks(pr_df):
    
    all_stock_vol = []
    numstocks_final = 0 #This stays 0 until changed by function

    # Performs the calculations for every column
    for pctchange in range (len(pr_df.columns)):
        # Calculates the standard deviation of the stock's daily percent return
        stock_vol = pr_df.iloc[:,pctchange].std()
        
        all_stock_vol.append(stock_vol)

    # Creates a dataframe that will store each stock and their volatility, sorts by non-decreasing volatility
    all_stock_vol_df = pd.DataFrame({"Stocks": pr_df.columns, "Daily % Returns Std": all_stock_vol,})
    all_stock_vol_df.sort_values(by = "Daily % Returns Std", inplace = True)
    all_stock_vol_df.reset_index(inplace = True)
    all_stock_vol_df.drop(columns = 'index', inplace = True)

    print("Final Selection of Stocks:")

    #Creates a list of the 'max_portfolios' least volatile stocks in the dataframe if there are more than 'max_portfolios', 
    #else creates a list of all of the stocks. Prints dataframe
    if len(all_stock_vol_df) > max_portfolios:
        final_portfolio_lst = all_stock_vol_df['Stocks'].loc[0 : max_portfolios + 1].values.tolist()
        numstocks_final = max_portfolios

        print(all_stock_vol_df.iloc[0 : max_portfolios + 1])
    else:
        final_portfolio_lst = all_stock_vol_df['Stocks'].values.tolist()
        numstocks_final = len(all_stock_vol_df)

        print(all_stock_vol_df)

    print(f"Stocks in Portfolio ({numstocks_final}): \n{final_portfolio_lst}")
    
    return final_portfolio_lst

ticklst_final = lowest_volatile_stocks(pr_df)

Final Selection of Stocks:
  Stocks  Daily % Returns Std
0      F             1.334880
1   APPL             1.435436
2    BAC             1.450386
3    JNJ             1.461897
4      T             1.527385
5     PG             1.916060
Stocks in Portfolio (6): 
['F', 'APPL', 'BAC', 'JNJ', 'T', 'PG']


In [10]:
#Piero's Note: not sure what this does. Ohh wait I think I did that above lol

# Consumes a list of tickers and a DataFrame. The requirements of the DataFrame are:
#   a) df is either a DataFrame of closing prices of stocks with the stock name as the column header or a DataFrame of percent returns of stocks 
#      with the stock name as the column header
#   b) df must include all of the tickers in list_of_ticks
# The function returns a dataframe of the stocks from list_of_ticks and either their closing prices or daily percent returns, depending on what 
# was inputted, with the stock name as the column header
def lowest_volatile_stocks_df_filter(list_of_ticks, df):
    # Creates the dataframe to store the data and creates the first column with the first stock's data from list_of_ticks
    final_df = pd.DataFrame({list_of_ticks[0]: df[list_of_ticks[0]],})

    # Creates a new column with the stock's data and the stock name as the header for every ticker in list_of_ticks, excluding the first ticker
    for i in range(1,25):
        final_df[list_of_ticks[i]] = df[list_of_ticks[i]]

    return final_df

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Piero Camposeo, Sathun Suthakaran, Ishaan Bansal